# Reference

- https://zhuanlan.zhihu.com/p/43289968
- https://dev.mysql.com/doc/connector-python/en/connector-python-example-ddl.html
- https://www.w3schools.com/python/python_mysql_create_db.asp
- https://www.w3resource.com/mysql/mysql-tutorials.php

# Connect to mysql server

In [1]:
# Connect to mysql

import mysql.connector
from mysql.connector import errorcode

try:
    cnx = mysql.connector.connect(user='root', password="Rui@2492592", host="127.0.0.1") #cnx is db object
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)

In [2]:
cursor = cnx.cursor()

# Create Database

In [5]:
DB_NAME = "school"

def create_database(cursor):
    try:
        cursor.execute("CREATE DATABASE {}".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creat database {}".format(DB_NAME))
        exit(1)

try:
    cursor.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cursor.execute("USE {}".format(DB_NAME))
        cnx.database = DB_NAME # After create database, you need assign ton connector
    else:
        print(err)
        exit(1)    

# Show Databases

In [3]:
cursor.execute("SHOW DATABASES")
for x in cursor:
    print(x)

('classicmodels',)
('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('school',)
('sys',)
('world',)


# Drop Database

In [25]:
cursor.execute("DROP DATABASE IF EXISTS python_mysql")

# Show current database

In [6]:
cursor.execute("SELECT DATABASE()")
for x in cursor.fetchall():
    print(x)

('school',)


# Create Table
- tip: the table name need to around in ``.

In [24]:
TABLES = {}

TABLES['Student'] = (
    " CREATE TABLE `Student`(" # need brackcet every row
    " `s_id` VARCHAR(20),"
    " `s_name` VARCHAR(20) NOT NULL DEFAULT '',"
    " `s_birth` VARCHAR(20) NOT NULL DEFAULT '',"
    " `s_sex` VARCHAR(20) NOT NULL DEFAULT '',"
    " PRIMARY KEY (`s_id`))")

TABLES['Course'] = (
    " CREATE TABLE `Course`("
    " `c_id` VARCHAR(20),"
    " `c_name` VARCHAR(20) NOT NULL DEFAULT '',"
    " `t_id` VARCHAR(20) NOT NULL,"
    " PRIMARY KEY (`c_id`))")

TABLES['Teacher'] = (
"CREATE TABLE `Teacher` ("
"`t_id` VARCHAR(20),"
"`t_name` VARCHAR(20) NOT NULL DEFAULT '',"
"PRIMARY KEY (`t_id`))")

TABLES['Score'] = (
"CREATE TABLE `Score` ("
"`s_id` VARCHAR(20),"
"`c_id` VARCHAR(20),"
"`s_score` INT(3),"
"PRIMARY KEY (`s_id`,`c_id`))")

In [25]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Create table {}".format(table_name),end=' ')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("talbe {} already exist".format(table_name))
        else:
            print(err.msg)
    else:
        print("OK")

Create table Student talbe Student already exist
Create table Course talbe Course already exist
Create table Teacher OK
Create table Score OK


# Show table in current database

In [7]:
cursor.execute("SHOW TABLES")
for x in cursor.fetchall():
    print(x)

('course',)
('score',)
('student',)
('teacher',)


# Insert Data

- you must commit the data after a sequence of INSERT, DELETE, and UPDATE statements.

In [55]:
add_student = ("INSERT INTO Student"
              "(s_id, s_name, s_birth, s_sex)"
              "VALUES (%s, %s, %s, %s)" )

add_course = ("INSERT IGNORE INTO Course"
             "(c_id, c_name, t_id)"
             "VALUES (%s, %s, %s)" )

add_teacher = ("INSERT INTO Teacher"
              "(t_id, t_name)"
              "VALUES (%s, %s)")

add_score = ("INSERT INTO Score"
            "(s_id, c_id, s_score)"
            "Values (%s, %s, %s)")

data_student = [('01' , '赵雷' , '1990-01-01' , '男'),
               ('02' , '钱电' , '1990-12-21' , '男'),
               ('03' , '孙风' , '1990-05-20' , '男'),
               ('04' , '李云' , '1990-08-06' , '男'),
               ('05' , '周梅' , '1991-12-01' , '女'),
               ('06' , '吴兰' , '1992-03-01' , '女'),
               ('07' , '郑竹' , '1989-07-01' , '女'),
               ('08' , '王菊' , '1990-01-20' , '女')]

data_course = [('01' , '语文' , '02'),
              ('02' , '数学' , '01'),
              ('03' , '英语' , '03')]

data_teacher = [('01' , '张三'),
               ('02' , '李四'),
               ('03' , '王五')]

data_score = [('01' , '01' , 80),
             ('01' , '02' , 90),
             ('01' , '03' , 99),
             ('02' , '01' , 70),
             ('02' , '02' , 60),
             ('02' , '03' , 80),
             ('03' , '01' , 80),
             ('03' , '02' , 80),
             ('03' , '03' , 80),
             ('04' , '01' , 50),
             ('04' , '02' , 30),
             ('04' , '03' , 20),
             ('05' , '01' , 76),
             ('05' , '02' , 87),
             ('06' , '01' , 31),
             ('06' , '03' , 34),
             ('07' , '02' , 89),
             ('07' , '03' , 98)]

In [42]:
# Insert student
cursor.executemany(add_student, data_student)
cnx.commit() 
print(cursor.rowcount, "was inserted.")

8 was inserted.


In [48]:
# Insert course
cursor.executemany(add_course, data_course)
cnx.commit()
print(cursor.rowcount, "was inserted.")

0 was inserted.


In [50]:
# Insert teacher
cursor.executemany(add_teacher, data_teacher)
cnx.commit()
print(cursor.rowcount, "was inserted.")

3 was inserted.


In [56]:
# Insert score
cursor.executemany(add_score, data_score)
cnx.commit()
print(cursor.rowcount, "was inserted.")

18 was inserted.


# Show table content

In [8]:
cursor.execute("SELECT * FROM Student")
for x in cursor.fetchall():
    print(x)

('01', '赵雷', '1990-01-01', '男')
('02', '钱电', '1990-12-21', '男')
('03', '孙风', '1990-05-20', '男')
('04', '李云', '1990-08-06', '男')
('05', '周梅', '1991-12-01', '女')
('06', '吴兰', '1992-03-01', '女')
('07', '郑竹', '1989-07-01', '女')
('08', '王菊', '1990-01-20', '女')


In [10]:
cursor.execute("SELECT * FROM Course")
for x in cursor.fetchall():
    print(x)

('01', '语文', '02')
('02', '数学', '01')
('03', '英语', '03')


In [11]:
cursor.execute("SELECT * FROM Teacher")
for x in cursor.fetchall():
    print(x)

('01', '张三')
('02', '李四')
('03', '王五')


In [12]:
cursor.execute("SELECT * FROM Score")
for x in cursor.fetchall():
    print(x)

('01', '01', 80)
('01', '02', 90)
('01', '03', 99)
('02', '01', 70)
('02', '02', 60)
('02', '03', 80)
('03', '01', 80)
('03', '02', 80)
('03', '03', 80)
('04', '01', 50)
('04', '02', 30)
('04', '03', 20)
('05', '01', 76)
('05', '02', 87)
('06', '01', 31)
('06', '03', 34)
('07', '02', 89)
('07', '03', 98)


In [9]:
def run_query(query):
    cursor.execute(query)
    for x in cursor.fetchall():
        print(x)

# Q1 
- 查询课程编号为“01”的课程比“02”的课程成绩高的所有学生的学号（重点）

In [15]:
# 构建三个表， 答案其实是查询了学生的所有个人信息，不止学号
query = ("SELECT st.*, a.s_score, b.s_score " # nothic the white space before "
        "FROM Student st "
        "INNER JOIN (SELECT s_id, s_score FROM Score WHERE c_id = '01') a ON st.s_id = a.s_id "
        "INNER JOIN (SELECT s_id, s_score FROM Score WHERE c_id = '02') b ON a.s_id = b.s_id "
        "WHERE a.s_score > b.s_score")

cursor.execute(query)
for x in cursor.fetchall():
    print(x)

('02', '钱电', '1990-12-21', '男', 70, 60)
('04', '李云', '1990-08-06', '男', 50, 30)


In [16]:
# 只查询学号

query = ("SELECT a.s_id, a.s_score, b.s_score "
        "FROM (SELECT s_id, s_score FROM Score WHERE c_id = '01') a "
        "INNER JOIN (SELECT s_id, s_score FROM Score WHERE c_id = '02') b ON a.s_id = b.s_id "
        "WHERE a.s_score > b.s_score")

cursor.execute(query)
for x in cursor.fetchall():
    print(x)

('02', 70, 60)
('04', 50, 30)


# Q2
- 查询平均成绩大于60分的学生的学号和平均成绩

In [10]:
query = ("SELECT sc.s_id, AVG(sc.s_score) "
        "FROM Score sc "
        "GROUP BY sc.s_id HAVING AVG(sc.s_score) > 60")

run_query(query)

('01', Decimal('89.6667'))
('02', Decimal('70.0000'))
('03', Decimal('80.0000'))
('05', Decimal('81.5000'))
('07', Decimal('93.5000'))


# Q2-2
- 查询平均成绩小于60分的学生的学号,姓名和平均成绩

In [20]:
query = ("SELECT st.s_id, st.s_name, AVG(IFNULL(sc.s_score,0)) as avg "
        "FROM Student st "
        "LEFT JOIN Score sc "
        "ON st.s_id = sc.s_id "
        "GROUP BY st.s_id "
        "HAVING AVG(sc.s_score) IS NULL OR AVG(sc.s_score)< 60 ")

run_query(query)

('04', '李云', Decimal('33.3333'))
('06', '吴兰', Decimal('32.5000'))
('08', '王菊', Decimal('0.0000'))


# Q3
- 查询所有学生的学号、姓名、选课数、总成绩（不重要）

In [22]:
query = ("SELECT st.s_id, st.s_name, COUNT(sc.c_id) AS CourseNum, SUM(sc.s_score) AS TotalScore "
        "FROM Student st "
        "LEFT JOIN Score sc "
        "ON st.s_id = sc.s_id "
        "GROUP BY st.s_id")

run_query(query)

('01', '赵雷', 3, Decimal('269'))
('02', '钱电', 3, Decimal('210'))
('03', '孙风', 3, Decimal('240'))
('04', '李云', 3, Decimal('100'))
('05', '周梅', 2, Decimal('163'))
('06', '吴兰', 2, Decimal('65'))
('07', '郑竹', 2, Decimal('187'))
('08', '王菊', 0, None)


# Q4
- 查询姓“猴”的老师的个数

In [24]:
query = ("SELECT COUNT(t_id) "
        "FROM Teacher "
        "WHERE t_name LIKE '张%'")

run_query(query)

(1,)


# Q5
- 查询没学过“张三”老师课的学生的学号、姓名

In [26]:
query = ("SELECT st.s_id, st.s_name "
        "FROM Student st "
        "WHERE st.s_id NOT IN "
        "(SELECT sc.s_id FROM Score sc "
        "LEFT JOIN Course co ON sc.c_id = co.c_id "
        "LEFT JOIN Teacher te ON co.t_id = te.t_id "
        "WHERE te.t_name = '张三')")

run_query(query)

('06', '吴兰')
('08', '王菊')


# Q6
- 查询学过“张三”老师所教的所有课的同学的学号、姓名

In [34]:
query = ("SELECT st.s_id, st.s_name "
        "FROM Student st "
        "WHERE st.s_id IN "
        "(SELECT sc.s_id FROM Score sc "
        "LEFT JOIN Course co ON sc.c_id = co.c_id "
        "LEFT JOIN Teacher te ON co.t_id = te.t_id "
        "WHERE te.t_name = '张三' )")

run_query(query)

('01', '赵雷')
('02', '钱电')
('03', '孙风')
('04', '李云')
('05', '周梅')
('07', '郑竹')


# Q7

- 查询学过编号为“01”的课程并且也学过编号为“02”的课程的学生的学号、姓名（重点）

In [38]:
query = ("SELECT st.s_id, st.s_name "
        "FROM Student st "
        "WHERE st.s_id IN "
        "(SELECT a.s_id FROM "
        "(SELECT s_id, c_id FROM Score WHERE c_id = '01') a "
        "INNER JOIN (SELECT s_id, c_id FROM Score WHERE c_id = '02') b "
        "ON a.s_id = b.s_id)" )

run_query(query)

('01', '赵雷')
('02', '钱电')
('03', '孙风')
('04', '李云')
('05', '周梅')


# Q8
- 查询课程编号为“02”的总成绩

In [39]:
query = ("SELECT SUM(s_score) "
        "FROM Score "
        "WHERE c_id = '02'")

run_query(query)

(Decimal('436'),)


# Q9
- 查询所有课程成绩小于60分的学生的学号、姓名
- same as Q2

# Q10
- 查询没有学全所有课的学生的学号、姓名

In [44]:
query = ("SELECT st.s_id, st.s_name, COUNT(sc.c_id) "
        "FROM Student st "
        "LEFT JOIN Score sc "
        "ON st.s_id = sc.s_id "
        "GROUP BY st.s_id "
        "HAVING COUNT(sc.c_id) < (SELECT COUNT(DISTINCT c_id) FROM Course)")

run_query(query)

('05', '周梅', 2)
('06', '吴兰', 2)
('07', '郑竹', 2)
('08', '王菊', 0)


# Q11
- 查询至少有一门课与学号为“01”的学生所学课程相同的学生的学号和姓名（重点）

In [46]:
# Method One: s_id from student in the condition
# query = ("SELECT DISTINCT st.s_id, st.s_name "
#         "FROM Student st "
#         "WHERE st.s_id IN "
#         "(SELECT sc.s_id "
#         "FROM Score sc "
#         "WHERE c_id IN (SELECT c_id FROM Score WHERE s_id = '01') AND sc.s_id != '01' )")

# Method Two: JOIN
query = ("SELECT DISTINCT st.s_id, st.s_name "
        "FROM Student st "
        "INNER JOIN Score sc "
        "ON st.s_id = sc.s_id "
        "WHERE sc.c_id IN (SELECT c_id FROM Score WHERE s_id = '01')AND sc.s_id != '01' ")

run_query(query)

('02', '钱电')
('03', '孙风')
('04', '李云')
('05', '周梅')
('06', '吴兰')
('07', '郑竹')


# Q12
- 查询和“01”号同学所学课程完全相同的其他同学的学号(重点)

In [ ]:
# Group_concat https://www.w3resource.com/mysql/mysql-tutorials.php
query = ("SELECT st.s_id, st.s_name"
        "FROM Student st "
        "INNER JOIN Score sc ON st.s_id = sc.s_id "
        "GROUP BY sc.s_id HAVING GROUP_CONCAT()")

# Delete all row 

In [39]:
cursor.execute("DELETE FROM Student")
cnx.commit()

# Disconnect with mysql server

In [59]:
cursor.close()
cnx.close()